## PIL module documentation

The PIL module contains a polarity inversion line detector class and a parallelized implementation of the detector class. The detector class can be applied to image data or video data, and output a selective set of polarity inversion line for the image or the video.


### PIL detector class

_class_ PIL.__PIL__(polar_threshold=0.90,window_size=5,edge_retain=0.003,radius=12,min_samples=5)

Apply a Polarity Inversion Line(PIL) detection statistical algorithm over 2D vertical magnetic field image data. The algorithm consists of finding strong polar magnitude threshold, filtering pixels with local strong positive and negative polars, scharr edge detection and DBSCAN pixels clustering. Given an image, the algorithm is able to locate, at user's discretion, a certain number of Polarity Inversion Line segments.

Read more in the PIL module introduction notebook.
   

__Parameters__:

__polar__\_ __threshold__:  ___float, optional, default: 0.90, ranged from 0 to 1___

The quantile threshold for a pixel with positive/negative vertical magnetic field to be considered as a "strong" pixel. All positive/negative pixels of the image are ranked in an ascending order based on the magnitude of the vertical magnetic field. And only pixels lying on the top quantile will be considered as "strong" pixels. Setting the threshold close to 1 means only defining the very top few pixels with the largest vertical magnetic field component. 

__window__\_ __size__: ___Positive Int, optional, default: 5, typically odd___

The size of an arbitrary pixel's neighborhood. For each pixel, its neighborhood is defined as a square shape subimage of the original image that has the pixel of interest locating at the center of the subimage. Window_size is the length of the side of the subimage. This subimage includes several adjacent pixels which are considered as the "neighbors" of the pixel. Only pixels with "strong" positive and negative pixels in its neighborhood will be considered as candidates for polarity inversion line.


__edge__\_ __retain__: ___float, optional, default: 0.003, ranged from 0 to 1___
    
The fraction of high gradient pixels to retain for drawing the polarity inversion line. After filtering the pixels with local strong positive/negative neighbors, a scharr edge detection operator is applied to the filtered image, and each pixel has its gradient calculated. Edge_retain specifies the fraction of pixels with the highest gradients to retain for finally drawing the polarity inversion line.

__radius__: ___Int, optional, default: 12, positive___

The maximum distance for two pixels to be considered as neighbors of each other on the same polarity inversion line segment. After edge detection, the algorithm finally filters a set of pixels to draw the polarity inversion line. For two points to be considered as consisting of the same PIL, they must be reachable from each other in the sense that there is a path of core points along which any two points on the path has a distance less than radius. See _sklearn.cluster.DBSCAN_ __eps__ parameter for detail.

__min_samples__: ___Int, optional, default: 5, positive___

The number of pixels in a neighborhood for a pixel to be considered as a core point in the DBSCAN clustering algorithm. This includes the point itself. Basically, any set of pixels that can be considered as consisting of a polarity inversion line should have at least 1 core point. For the definition of core point, see the associated PIL module introduction notebook or  _sklearn.cluster.DBSCAN_.

__Attributes:__

__pos__\_ __threshold__: ___float___

The threshold for a positive pixel to be considered as a strong positive pixel.

__neg__\_ __threshold__: ___float___

The threshold for a negative pixel to be considered as a strong negative pixel.

**N**: ___Int___

The number of polarity inversion line segments founded.

__Methods__:

__.fit(_self,data,select=1,global_\_ _threshold=0.3,mode='image'_)__
 
__Parameters:__

 * __data__: ___Array of shape [height,width], or array of shape [video___\_ ___length,height,width]___.
   
   Input image data or video data. Missing values allowed.
   
   
 * __select__: ___Int, optional, default: 1___ 
 
   Maximum number of PIL segments to identify per image. 
   
 *   

__.Image_Decompose(self,image)__ 
 
 Check if image can be processed by the algorithm and decompose image into a positive component and a negative component, and find the threshold for strong positive/negative polars.

__.video_data_decompose(self,video)__
 
 Check if a video can be processed by the algorithm and decompose video into a positive component and a negative component, and find the threshold for strong positive/negative polars throughout the whole video.